In [1]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px

### Compute ressources

In [2]:
# Ncores
N_nodes = 24
cores_per_node = 64
N_cores = N_nodes * cores_per_node
print(f"Total number of cores available: {N_cores}")

Total number of cores available: 1536


### Processing time in deg2 per core hours

In [3]:
# IMCOM
# Time per filter per deg2 in hours
imcom_time = 11_000

# Metadetection
# Time per deg2 in hours
mdet_time = 100

### Survey area in deg2

In [4]:
# Deep field
deep_field_area = 20
deep_field_n_filters = 3
obs_end_date = "2027-02-10"  # YYYY-MM-DD

# Medium tier
medium_tier_area = 200
medium_tier_n_filters = 3

### Analysis Planning

In [5]:
# We will likely do a cycle of processing and analysis before doing the processing of the medium tier
analysis_time = 30  # days
n_cycles = 3

### Timeline

In [6]:
tasks_list = []
for i in range(n_cycles):
    if i == 0:
        deps = None
    else:
        deps = f"Analysis_{i}"
    task = {
        "task": "Processing Deep Field",
        "task_id": f"Processing_{i+1}",
        "duration_days": (imcom_time * deep_field_n_filters * deep_field_area + mdet_time * deep_field_area) / N_cores / 24,
        "depends_on": deps,
        "start": None,
        "end": None,
    }
    tasks_list.append(task)
    task = {
        "task": "Analysis Deep Field",
        "task_id": f"Analysis_{i+1}",
        "duration_days": analysis_time,
        "depends_on": f"Processing_{i+1}",
        "start": None,
        "end": None,
    }
    tasks_list.append(task)
task = {
    "task": "Processing Medium Tier",
    "task_id": "Processing_Medium_Tier",
    "duration_days": (imcom_time * medium_tier_n_filters * medium_tier_area + mdet_time * medium_tier_area) / N_cores / 24,
    "depends_on": f"Analysis_{n_cycles}",
    "start": None,
    "end": None,
}
tasks_list.append(task)
task = {
    "task": "Initial Science on Deep Field",
    "task_id": "Initial Science on Deep Field",
    "duration_days": 6 * 30,
    "depends_on": f"Analysis_{n_cycles}",
    "start": None,
    "end": None,
}
tasks_list.append(task)
df = pd.DataFrame(tasks_list)

In [7]:
df

,task,task_id,duration_days,depends_on,start,end
0,Processing Deep Field,Processing_1,17.957899,None,None,None
1,Analysis Deep Field,Analysis_1,30.000000,Processing_1,None,None
2,Processing Deep Field,Processing_2,17.957899,Analysis_1,None,None
3,Analysis Deep Field,Analysis_2,30.000000,Processing_2,None,None
4,Processing Deep Field,Processing_3,17.957899,Analysis_2,None,None
5,Analysis Deep Field,Analysis_3,30.000000,Processing_3,None,None
6,Processing Medium Tier,Processing_Medium_Tier,179.578993,Analysis_3,None,None
7,Initial Science on Deep Field,Initial Science on Deep Field,180.000000,Analysis_3,None,None


In [8]:
start_date = datetime.strptime(obs_end_date, "%Y-%m-%d")

# Compute start/end dates
task_start = {}
task_end = {}

for _, row in df.iterrows():
    dep = row["depends_on"]
    if dep:
        start_time = task_end[dep]
    else:
        start_time = start_date  # independent task
    end_time = start_time + timedelta(days=row["duration_days"])
    
    task_start[row["task_id"]] = start_time
    task_end[row["task_id"]] = end_time


df["start"] = df["task_id"].map(task_start)
df["end"] = df["task_id"].map(task_end)

In [9]:
df

,task,task_id,duration_days,depends_on,start,end
0,Processing Deep Field,Processing_1,17.957899,None,2027-02-10 00:00:00.000,2027-02-27 22:59:22.500
1,Analysis Deep Field,Analysis_1,30.000000,Processing_1,2027-02-27 22:59:22.500,2027-03-29 22:59:22.500
2,Processing Deep Field,Processing_2,17.957899,Analysis_1,2027-03-29 22:59:22.500,2027-04-16 21:58:45.000
3,Analysis Deep Field,Analysis_2,30.000000,Processing_2,2027-04-16 21:58:45.000,2027-05-16 21:58:45.000
4,Processing Deep Field,Processing_3,17.957899,Analysis_2,2027-05-16 21:58:45.000,2027-06-03 20:58:07.500
5,Analysis Deep Field,Analysis_3,30.000000,Processing_3,2027-06-03 20:58:07.500,2027-07-03 20:58:07.500
6,Processing Medium Tier,Processing_Medium_Tier,179.578993,Analysis_3,2027-07-03 20:58:07.500,2027-12-30 10:51:52.500
7,Initial Science on Deep Field,Initial Science on Deep Field,180.000000,Analysis_3,2027-07-03 20:58:07.500,2027-12-30 20:58:07.500


In [10]:
fig = px.timeline(df, x_start="start", x_end="end", y="task", color="task", title="Project Timeline")
# fig.update_yaxes(autorange="reversed")  # Gantt-style
fig.show()

In [28]:
print(
    "Summary\n"
    "-------\n"
    f"N cores: {N_cores}\n"
    "\n"
    f"Deep field area: {deep_field_area} deg2\n"
    f"N filters: {deep_field_n_filters}\n"
    f"Deep field Processing start date: {task_start['Processing_1'].strftime('%Y-%m-%d')}\n"
    f"Deep field Processing time: {round((task_end['Processing_1'] - task_start['Processing_1']).total_seconds() / 3600 / 24, 2)} days\n"
    "\n"
    f"Medium tier area: {medium_tier_area} deg2\n"
    f"N filters: {medium_tier_n_filters}\n"
    f"Medium Processing start date: {task_start['Processing_Medium_Tier'].strftime('%Y-%m-%d')}\n"
    f"Medium Processing end date: {task_end['Processing_Medium_Tier'].strftime('%Y-%m-%d')}\n"
    f"Medium Processing duration: {round((task_end['Processing_Medium_Tier'] - task_start['Processing_Medium_Tier']).total_seconds() / 3600 / 24, 2)} days\n"
)

Summary
-------
N cores: 1536

Deep field area: 20 deg2
N filters: 3
Deep field Processing start date: 2027-02-10
Deep field Processing time: 17.96 days

Medium tier area: 200 deg2
N filters: 3
Medium Processing start date: 2027-07-03
Medium Processing end date: 2027-12-30
Medium Processing duration: 179.58 days



In [22]:
(task_end['Processing_1'] - task_start['Processing_1']).total_seconds() / 3600 / 24

17.957899305555554